In [1]:
import mne

#from sklearnex import patch_sklearn

#patch_sklearn()

from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 

import numpy as np
import os
from tqdm.auto import tqdm

In [4]:
output_dir = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/output"
subject = ["ap"]

In [14]:
clean_scores = list()
noisy_scores = list()

for subject in tqdm(["ap", "ca", "ha", "ja", "mv", "wc", "zt"], desc="Subject"):
    tfr = mne.time_frequency.read_tfrs(os.path.join(output_dir, "tfr", f"{subject}-tfr.h5"))[0]

    X_clean, X_noisy = tfr["clean"]._data.astype(np.float32), tfr["noisy"]._data.astype(np.float32)
    y_clean, y_noisy = tfr["clean"].events[:, -1], tfr["noisy"].events[:, -1] # Labels are in the last col

    del tfr

    y_clean, y_noisy = LabelEncoder().fit_transform(y_clean), LabelEncoder().fit_transform(y_noisy)

    pipe = make_pipeline(
        #mne.decoding.Scaler(scalings="median"),
        mne.decoding.Vectorizer(),
        RobustScaler(),
        #SVC(kernel="liblinear")
        LogisticRegression(solver="liblinear")
    )

    gle = mne.decoding.GeneralizingEstimator(pipe, scoring="balanced_accuracy")

    clean_scores.append(np.mean(mne.decoding.cross_val_multiscore(gle, X_clean, y_clean)))

    gle.fit(X_clean, y_clean)
    noisy_scores.append(gle.score(X_noisy, y_noisy))

    del X_clean, X_noisy, y_clean, y_noisy

Subject:   0%|          | 0/7 [00:00<?, ?it/s]

Reading /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/output/tfr/ap-tfr.h5 ...
Not setting metadata


  0%|          | Fitting GeneralizingEstimator : 0/51 [00:00<?,       ?it/s]

  0%|          | Scoring GeneralizingEstimator : 0/2601 [00:00<?,       ?it/s]

KeyboardInterrupt: 

In [11]:
np.mean(clean_scores)

0.6798771121351767

In [13]:
np.mean(noisy_scores)

0.9563748079877111

In [16]:
tfr = mne.time_frequency.read_tfrs(os.path.join(output_dir, "tfr", f"{subject}-tfr.h5"))[0]

Reading /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/output/tfr/ap-tfr.h5 ...
Not setting metadata


In [17]:
tfr.event_id

{'face/clean': 10001,
 'face/noisy': 10001,
 'house/clean': 10002,
 'house/noisy': 10002}